In [ ]:
import tensorflow as tf

# Configure GPU memory usage
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        # Currently, memory growth needs to be the same across GPUs
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        logical_gpus = tf.config.experimental.list_logical_devices('GPU')
        print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
    except RuntimeError as e:
        # Memory growth must be set before GPUs have been initialized
        print(e)

# Session configuration
config = tf.compat.v1.ConfigProto()
config.gpu_options.allow_growth = True
config.log_device_placement = True
sess = tf.compat.v1.Session(config=config)
tf.compat.v1.keras.backend.set_session(sess)


In [ ]:
import cv2
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import random
import tensorflow as tf
from keras import layers, models
from keras.callbacks import EarlyStopping
from sklearn.model_selection import train_test_split
from keras.models import load_model
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, roc_curve, roc_auc_score



In [ ]:
pic=plt.imread("/kaggle/input/state-farm-distracted-driver-detection/imgs/train/c8/img_100814.jpg")
plt.imshow(pic, cmap="gray")
print(pic.shape)

In [ ]:
DataDirectory = '/kaggle/input/state-farm-distracted-driver-detection/imgs/train'
TestDirectory= '/kaggle/input/state-farm-distracted-driver-detection/imgs/test'
Classes = ['c0','c1','c2','c3','c4','c5','c6','c7','c8','c9']
img_size = 240

In [ ]:
def create_training_data(Classes, Datadirectory, img_size, max_images_per_class=1500):
    training_data = []
    for category in Classes:
        path = os.path.join(Datadirectory, category)
        class_num = Classes.index(category)
        image_count = 0  # Initialize a counter for images in the class

        for img in os.listdir(path):
            if image_count >= max_images_per_class:
                break  # Stop processing after reaching the maximum number of images per class
            
            img_array = cv2.imread(os.path.join(path, img), cv2.IMREAD_GRAYSCALE)
            
            # Resize the image while maintaining the aspect ratio
            desired_size = (img_size, img_size)
            height, width = img_array.shape
            aspect_ratio = width / height

            if aspect_ratio >= 1:
                new_width = desired_size[0]
                new_height = int(new_width / aspect_ratio)
            else:
                new_height = desired_size[1]
                new_width = int(new_height * aspect_ratio)

            resized_image = cv2.resize(img_array, (new_width, new_height))

            # Pad the resized image to make it square (img_size x img_size)
            pad_width = (desired_size[1] - new_height) // 2
            pad_height = (desired_size[0] - new_width) // 2
            padded_image = np.pad(resized_image, ((pad_width, pad_width), (pad_height, pad_height)), mode='constant', constant_values=0)

            # Convert the grayscale image to RGB
            rgb_image = cv2.cvtColor(padded_image, cv2.COLOR_GRAY2RGB)
            
            training_data.append([rgb_image, class_num])
            image_count += 1  # Increment the counter for images in the class

    print("Data processing completed.")
    return training_data

In [ ]:
import os
import cv2
import numpy as np

def create_testing_data(TestDirectory, img_size, max_images=8000):
    testing_data = []
    path = os.path.join(TestDirectory, '')
    count = 0  # Initialize a counter

    for img in os.listdir(path):
        if count >= max_images:
            break  # Stop processing after reaching the maximum number of images
        
        img_array = cv2.imread(os.path.join(path, img), cv2.IMREAD_GRAYSCALE)
        
        # Resize the image while maintaining the aspect ratio
        desired_size = (img_size, img_size)
        height, width = img_array.shape
        aspect_ratio = width / height

        if aspect_ratio >= 1:
            new_width = desired_size[0]
            new_height = int(new_width / aspect_ratio)
        else:
            new_height = desired_size[1]
            new_width = int(new_height * aspect_ratio)

        resized_image = cv2.resize(img_array, (new_width, new_height))

        # Pad the resized image to make it square (img_size x img_size)
        pad_width = (desired_size[1] - new_height) // 2
        pad_height = (desired_size[0] - new_width) // 2
        padded_image = np.pad(resized_image, ((pad_width, pad_width), (pad_height, pad_height)), mode='constant', constant_values=0)

        # Convert the grayscale image to RGB
        rgb_image = cv2.cvtColor(padded_image, cv2.COLOR_GRAY2RGB)

        testing_data.append([rgb_image])
        count += 1  # Increment the counter

    print("Data processing completed.")
    return testing_data


In [ ]:
training_data= create_training_data(Classes, DataDirectory, img_size)

In [ ]:
testing_data= create_testing_data(TestDirectory, img_size)

In [ ]:
print(len(training_data))
print(len(testing_data))

In [ ]:
X = []
y = []
for features, label in training_data:
    X.append(features)
    y.append(label)

X = np.array(X).reshape(-1, img_size, img_size, 3)
Y=np.array(y)

# Builidng the model

In [ ]:
# Define the input shape
input_shape = (240, 240, 3)
model = models.Sequential()

model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=input_shape))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.Conv2D(128, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(256, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(384, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(512, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(1024, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))

# Flatten the output from Conv2D layers
model.add(layers.Flatten())

# Add fully connected layers with specified parameters
model.add(layers.Dense(16384, activation='relu'))
model.add(layers.Dense(180, activation='relu'))
model.add(layers.Dense(32, activation='relu'))
model.add(layers.Dense(10,activation='softmax'))

In [ ]:
model.summary()

In [ ]:
model.compile(loss="binary_crossentropy", optimizer = "adam", metrics = ["accuracy"])

early_stopping_callback = EarlyStopping(monitor='val_accuracy', patience=4, mode='max', verbose=1)

X_train, X_val, Y_train, Y_val = train_test_split(X, Y, test_size=0.2, random_state=42, shuffle=True)

Y_train = np_utils.to_categorical(y_train,num_classes=10)
Y_val = np_utils.to_categorical(y_test,num_classes=10)

In [ ]:
history = model.fit(X_train, Y_train, epochs=20, batch_size=32, validation_data=(X_val, Y_val), callbacks=[early_stopping_callback])
